In [2]:
import polars as pl

In [3]:
train = pl.read_csv("../data/train.csv")
test = pl.read_csv("../data/test.csv")
era = pl.read_csv("../data/era.csv")
sample_submission = pl.read_csv("../data/sample_submission.csv")

In [4]:
train.head()

名称,棟名,種別2,国,時代,都道府県 ※美工品は「所有者住所（所在都道府県）」,所在地,保管施設の名称,所有者名,管理団体又は責任者,緯度,経度,is_kokuhou
str,str,str,str,str,str,str,str,str,str,f64,f64,i64
"""泉布観""",null,"""近代／文化施設""",null,"""明治""","""大阪府""","""大阪府大阪市北区天満橋一丁目…",null,"""大阪市""",null,34.69811,135.5214,0
"""絹本著色准胝仏母像""",null,"""絵画""","""日本""","""鎌倉""","""京都府（京都府）""",null,null,"""広隆寺""",null,null,null,0
"""石上神宮拝殿""",null,"""近世以前／寺院""",null,"""鎌倉前期""","""奈良県""","""奈良県天理市布留町""",null,"""石上神宮""",null,34.5978,135.852,1
"""剣〈銘真守（備前）／〉""",null,"""工芸品""","""日本""","""鎌倉""","""和歌山県（和歌山県）""",null,null,"""宗教法人金剛峯寺""","""財団法人高野山文化財保存会""",null,null,0
"""旧新町紡績所""","""工場本館""","""近代／産業・交通・土木""",null,"""明治""","""群馬県""","""群馬県高崎市新町二三三〇番地…",null,"""クラシエフーズ株式会社""",null,36.27766,139.1003,0


In [5]:
train.get_column("棟名").value_counts(sort=True)

棟名,counts
str,u32
null,5278
"""主屋""",88
"""表門""",34
"""本殿""",31
"""土蔵""",21
"""拝殿""",18
"""本堂""",15
"""米蔵""",15
"""文庫蔵""",13


In [6]:
train.get_column("種別2").value_counts(sort=True)

種別2,counts
str,u32
"""彫刻""",1148
"""工芸品""",1011
"""絵画""",860
"""書跡・典籍""",785
"""近世以前／寺院""",529
"""近世以前／神社""",522
"""近世以前／民家""",365
"""古文書""",324
"""考古資料""",298


In [7]:
train.get_column("国").value_counts(sort=True)

国,counts
str,u32
"""日本""",4014
null,2277
"""中国""",350
"""朝鮮""",34
"""中国・日本""",9
"""西洋""",8
"""イギリス""",3
"""オランダ""",2
"""ヴェトナム""",2


In [8]:
# データの整備を行う
train.get_column("時代").value_counts(sort=True)

時代,counts
str,u32
"""鎌倉""",1319
"""平安""",1045
"""桃山""",345
"""明治""",336
"""江戸中期""",296
"""室町""",295
"""江戸前期""",283
"""南北朝""",282
"""江戸""",237


In [9]:
train.get_column("保管施設の名称").value_counts(sort=True)

保管施設の名称,counts
str,u32
null,5062
"""東京国立博物館""",358
"""京都国立博物館""",146
"""奈良国立博物館""",63
"""天理大学附属天理図書館""",37
"""公益財団法人静嘉堂""",35
"""財団法人エム・オー・エー美術…",34
"""公益財団法人五島美術館""",32
"""国立歴史民俗博物館""",32


In [10]:
train.get_column("所有者名").value_counts(sort=True)

所有者名,counts
str,u32
null,523
"""独立行政法人国立文化財機構""",455
"""個人""",171
"""国（文化庁）""",97
"""法隆寺""",76
"""東大寺""",63
"""国(文部科学省)""",62
"""大山祇神社""",46
"""公益財団法人静嘉堂""",42


In [11]:
train.get_column("管理団体又は責任者").value_counts(sort=True)

管理団体又は責任者,counts
str,u32
null,6352
"""財団法人高野山文化財保存会""",94
"""姫路市""",41
"""鎌倉市""",25
"""岡山県""",15
"""宗教法人東福寺""",13
"""宗教法人相国寺""",11
"""宗教法人延暦寺""",10
"""宗教法人中尊寺""",9


In [12]:
train.get_column("時代").value_counts(sort=True)

時代,counts
str,u32
"""鎌倉""",1319
"""平安""",1045
"""桃山""",345
"""明治""",336
"""江戸中期""",296
"""室町""",295
"""江戸前期""",283
"""南北朝""",282
"""江戸""",237


In [13]:
test.get_column("時代").value_counts(sort=True)

時代,counts
str,u32
"""鎌倉""",2044
"""平安""",1625
"""明治""",520
"""桃山""",514
"""室町""",445
"""江戸前期""",427
"""南北朝""",407
"""江戸中期""",394
"""江戸""",388


In [15]:
train.get_column("所在地").value_counts(sort=True)

所在地,counts
str,u32
null,2832
"""東京国立博物館 東京都台東区…",322
"""京都国立博物館 京都府京都市…",107
"""奈良国立博物館 奈良県奈良市…",48
"""栃木県日光市山内""",43
"""兵庫県姫路市本町""",41
"""東京都世田谷区上野毛3-9-…",39
"""天理大学附属天理図書館 奈良…",38
"""東京都港区南青山6-5-1""",37
